In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [2]:
new_record = []
for seq_record in SeqIO.parse('/home/ouconstand/data/Virus_Host/seq_data/sequence_0.fasta','fasta'):
    seq = seq_record.seq
    if len(seq) > 2000:
        for i in range(0, len(seq), 500):
            if i + 2000 > len(seq):
                new_seq = seq[-2000:]
                rec = SeqRecord(new_seq, id=seq_record.id, description=seq_record.description, name=seq_record.name)
                new_record.append(rec)
                break

            new_seq = seq[i:i+2000]
            rec = SeqRecord(new_seq,id=seq_record.id,description=seq_record.description,name=seq_record.name)
            new_record.append(rec)

        else:
            print('sequence length is less than 2000')
            print(seq_record.description)

In [3]:
data = [i.seq for i in new_record]
labels = [i.id for i in new_record]

In [4]:
set(labels)

{'KT368856.1', 'KT368895.1', 'MT726043.1', 'MZ519413.1'}

In [5]:
import numpy as np

def nucl_to_onehot(sequence, dimension=5):
    __nucl_list = ['A', 'T', 'C', 'G', 'N']
    __nucl_to_int = dict((nucl, int) for int, nucl in enumerate(__nucl_list))
    __integer_encoded = [__nucl_to_int[base] for base in sequence]

    result = np.zeros((len(sequence), dimension))
    for i, base in enumerate(__integer_encoded):
        result[i, base] = 1
    return result


In [44]:
from keras.utils import to_categorical

labels_to_int = dict((int, label) for label, int in enumerate(set(labels)))
process_data = np.array([nucl_to_onehot(i) for i in data]).reshape(223,2000,5).astype('float32')
process_labels = to_categorical(np.array([labels_to_int[label] for label in labels]))


In [45]:
process_labels.shape

(223, 4)

In [46]:
process_data = process_data.astype('float32')
process_labels = process_labels.astype('float32')

In [47]:
process_data.shape

(223, 2000, 5)

In [70]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(512,activation='relu', input_shape=(2000, 5)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [71]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 2000, 512)         3072      
_________________________________________________________________
dense_28 (Dense)             (None, 2000, 256)         131328    
_________________________________________________________________
dense_29 (Dense)             (None, 2000, 4)           1028      
Total params: 135,428
Trainable params: 135,428
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.fit(process_data,process_labels,epochs=5)

Epoch 1/5


ValueError: in user code:

    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
   